# GRID_LRT Testbed Notebook

## 1. Setting up the Environment

The GRID LOFAR TOOLS have several infrastructure requirements. They are as follows:

1. ASTRON LOFAR staging credentials
2. PiCaS database access
3. Valid GRID proxy


Here, we'll test that all of the above are enabled and work:

In [1]:
import os
import GRID_LRT
print(GRID_LRT.__file__)
import subprocess
from GRID_LRT.get_picas_credentials import picas_cred
from GRID_LRT.Staging import stage_all_LTA
from GRID_LRT.Staging import state_all
from GRID_LRT.Staging import stager_access
from GRID_LRT import Token
pc=picas_cred()

/home/apmechev/test/lib/python2.7/site-packages/GRID_LRT-0.2-py2.7.egg/GRID_LRT/__init__.pyc
2018-01-17 13:34:08.535049 stager_access: Parsing user credentials from /home/apmechev/.awe/Environment.cfg
2018-01-17 13:34:08.535237 stager_access: Creating proxy


This should give a confirmation of that your LOFAR ASTRON credentials were properly read:

`2017-12-04 17:15:29.097902 stager_access: Parsing user credentials from /home/apmechev/.awe/Environment.cfg
2017-12-04 17:15:29.097973 stager_access: Creating proxy`

Next, we check that your PiCaS User and Database are set properly. You can also verify your password

In [2]:
print(pc.user)
print(pc.database)

apmechev
sksp_unittest


Next, we'll use the test srm.txt to show off our staging chops:

Stage the test srm.txt file. You'll get a StageID that you can use later.

In [3]:
os.path.exists('/home/apmechev/test/GRID_LRT/srm.txt')
stageID=stage_all_LTA.main('/home/apmechev/test/GRID_LRT/srm.txt') # NOTE! You'll get two emails every time you do this!
print(stageID)

Staging in Juleich
Setting up 1 srms to stage
staged with stageID  18405
18405


You can now re-run the cell below to check the current status of your staging request:

In [4]:
stage_all_LTA.get_stage_status(stageID) #crashes (py2.7?)
#The code below can also show you a more detailed status
statuses=stager_access.get_progress()

print(statuses)

()


In [5]:
statuses=stager_access.get_progress()
## When the staging completes, your stageID magically disappears from the database
# Neat, huh?
if not statuses:
    print "Staging status no longer in LTA Database" #This happens because bad programming
else:
    stager_access.prettyprint(statuses[str(stageID)])

Staging status no longer in LTA Database


You can also check the status of the srms two different ways (with srmls and with gfal)

In [178]:
## state_all.main('srm.txt') #gfal2 Doesn't work anymore?
for i in open('srm.txt','rb').readlines(): #Using srmls to loop over files ¯\_(ツ)_/¯
    out = subprocess.Popen(['srmls','-l',i],stdout=subprocess.PIPE).communicate()[0]
    print(i.split('/')[-1].strip()+" --> "+out.split('locality:')[1].split()[0])

L556776_SB230_uv.MS_ac189f93.tar --> ONLINE_AND_NEARLINE


## 2. Tokens! 
### 2. a) The manual way

Next we'll interface with PiCaS and start making tokens for our Observation:

here we need a string to link all the tokens in one Observation. We'll use the string 'demo_'+username in the sksp_dev database

In [185]:
uname = os.environ['USER']
th = Token.Token_Handler(t_type="demo_"+uname, uname=pc.user, pwd=pc.password, dbn='sksp_dev')

#Create the overview_view (has the number of todo, done, error, running, [...] tokens)
th.add_overview_view()

#Add the satus views (By default 'todo', 'locked', 'done', 'error')
th.add_status_views()

#Manually create a token:
manual_keys = {'manual_key':'manual_value','manual_int':1024}
man_token_1 = th.create_token(keys=manual_keys, append="manual") #will return the id of the manual token
print('manual_token_ID = ' + man_token_1)

manual_token_ID = t_demo_apmechev_manual


We can also manually create a Token with an automatic attachment:

In [186]:
manual_keys = {'manual_key':'manual_value','manual_int':0}
man_token_2 = th.create_token(keys=manual_keys, 
                            append="manual_with_attach",
                            attach=[open('srm.txt'),'srm_at_token_create.txt']) 

##We can also attach files after the token's been created:
th.add_attachment(man_token_2, open('srm.txt'), 'srm_added_later.txt')

#Double check that both files were attached. Returns a list of filenames:
man_2_attachies = th.list_attachments(man_token_2)
print("The two attached files are: "+str(man_2_attachies))

# We can also of course download attachments:
saved_attach=th.get_attachment(man_token_2,man_2_attachies[0],savename=man_2_attachies[0])
print("")
print('The attachemnt '+str(man_2_attachies[0])+" was saved at "+saved_attach)

assert(os.path.exists(saved_attach))
os.remove(saved_attach)
assert(not os.path.exists(saved_attach))


The two attached files are: ['srm_at_token_create.txt', 'srm_added_later.txt']

The attachemnt srm_at_token_create.txt was saved at /home/apmechev/test/GRID_LRT/srm_at_token_create.txt


We can also list the views and the tokens from each view:

In [187]:
print(th.views.keys()) #the views member of th is a dictionary of views 
locked_tokens = th.list_tokens_from_view('locked')

print(type(done_tokens)) #It's not a list!!
print("There are "+str(len(locked_tokens))+" 'locked' tokens")


todo_tokens = th.list_tokens_from_view('todo') 
# It's not a list because it procedurally pings CouchDB, ~generator
#Use the help below to browse how it works!!
##help(todo_tokens)

print("There are "+str(len(todo_tokens))+" 'todo' tokens")
print("")
print("They are:")
for i in todo_tokens:
    print("CouchDB token keys: "+str(i.keys()),"Token ID: "+i.id)


['todo', 'done', 'overview_total', 'locked', 'error']
<class 'GRID_LRT.couchdb.client.ViewResults'>
There are 0 'locked' tokens
There are 2 'todo' tokens

They are:
("CouchDB token keys: ['key', 'id', 'value']", 'Token ID: t_demo_apmechev_manual')
("CouchDB token keys: ['key', 'id', 'value']", 'Token ID: t_demo_apmechev_manual_with_attach')


You can set all tokens in a view to a Status, say 'locked'. This automatically locks the tokens!!

In [188]:

print('Lock status of the token: '+str(th.db[man_token_2]['lock'])+".")
print('Scrub count of the token: '+str(th.db[man_token_2]['scrub_count'])+".")
print("There are "+str(len(th.list_tokens_from_view('todo')))+" 'todo' tokens")
print("There are "+str(len(th.list_tokens_from_view('locked')))+" 'locked' tokens")
print("")
print("Setting status to locked for all todo tokens")
th.set_view_to_status(view_name='todo',status='locked') #Sets all todo tokens to "locked"

todo_tokens = th.list_tokens_from_view('todo') 
print("")

print("There are "+str(len(todo_tokens))+" 'todo' tokens")
### No more todo tokens!


locked_tokens = th.list_tokens_from_view('locked')
print("There are "+str(len(locked_tokens))+" 'locked' tokens")
##Now they're all locked!

print('Lock status of the token: '+str(th.db[man_token_2]['lock'])+".")
#You can reset all tokens from a view back to 'todo'. This increments the scrub_count field


resetted_tokens=th.reset_tokens('locked')
print("")
print("Resetting the locked tokens")
print('Scrub count of the token: '+str(th.db[man_token_2]['scrub_count'])+".")
print("There are "+str(len(th.list_tokens_from_view('todo')))+" 'todo' tokens")
print("There are "+str(len(th.list_tokens_from_view('locked')))+" 'locked' tokens")



Lock status of the token: 0.
Scrub count of the token: 0.
There are 2 'todo' tokens
There are 0 'locked' tokens

Setting status to locked for all todo tokens

There are 0 'todo' tokens
There are 2 'locked' tokens
Lock status of the token: 1.

Resetting the locked tokens
Scrub count of the token: 1.
There are 2 'todo' tokens
There are 0 'locked' tokens


Finally, you can create your own view. Views collect tokens that satisfy a certain boolean expression (where the token is referenced as 'doc'

For example: 

The todo view satsifies: `'doc.lock ==  0 && doc.done == 0 '` 

The locked view satisfies: `'doc.lock > 0 && doc.done == 0 '`

The done view satsifies: `'doc.status == "done" '`

In [189]:
th.add_view(v_name="demo_view",cond='doc.manual_int == 0 ') #Only one of our tokens has manual_int==0
print(th.views['demo_view']) #new view is here!

assert(len(th.list_tokens_from_view('demo_view'))==1)
print("There is "+str(len(th.list_tokens_from_view('demo_view')))+" tokens in the demo_view")

#Creating 2 more tokens for this view. If append isn't changed, the id is the same, so
#new tokens won't be created! But you can imagine a loop will make creation easy right?
_ = th.create_token(keys=manual_keys, 
                            append="manual_with_attach_1",  
                            attach=[open('srm.txt'),'srm_at_token_create.txt']) 
_ = th.create_token(keys=manual_keys, 
                            append="manual_with_attach_2",
                            attach=[open('srm.txt'),'srm_at_token_create.txt'])
print("There are "+str(len(th.list_tokens_from_view('demo_view')))+" tokens in the demo_view")
assert(len(th.list_tokens_from_view('demo_view'))==3)


<ViewDefinition '_design/demo_apmechev/_view/demo_view'>
There is 1 tokens in the demo_view
There are 3 tokens in the demo_view


Now we can delete all tokens in this view easily!

In [190]:
th.delete_tokens('demo_view')
assert(len(th.list_tokens_from_view('demo_view'))==0)
print("There are "+str(len(th.list_tokens_from_view('demo_view')))+" tokens in the demo_view")




Deleting Token t_demo_apmechev_manual_with_attach
Deleting Token t_demo_apmechev_manual_with_attach_1
Deleting Token t_demo_apmechev_manual_with_attach_2
There are 0 tokens in the demo_view


Now introducing Token Sets: Just an easy way to create tokens from a dictionary using a yaml file!

In [191]:
ts=Token.TokenSet(th=th)